# CS 109A
# Final Project- EDA Milestone 3

# Will Cho, Haruka Uchida, Jessica Zhao


Import libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
#import pydotplus
#import io
from sklearn.tree import export_graphviz
from IPython.display import Image
from IPython.display import display
from matplotlib import colors
import seaborn as sns
%matplotlib inline

In [2]:
#########################
### Read in Data ########
#########################

# if want to use ID number as the index, index_col = 'RID'
df = pd.read_csv('Data_Database/ADNIMERGE.csv')

#################################
### Get all Column Names ########
#################################

all_variables_first = list(df.columns.values)

#################################
### Make Dummy Variables ########
#################################

# there are probably more categoricals need to add to this array
categorical_col = ['DX', 'PTGENDER']
df = pd.get_dummies(df, columns=categorical_col)
# make list of all variables, including dummy variables
all_variables_dum = list(df.columns.values)
len(all_variables_dum)

97

In [3]:
#################################
### Basic Summary Stats #########
#################################

df.describe()

,RID,SITE,AGE,PTEDUCAT,APOE4,FDG,PIB,AV45,CDRSB,ADAS11,...,AV45_bl,Years_bl,Month_bl,Month,M,DX_CN,DX_Dementia,DX_MCI,PTGENDER_Female,PTGENDER_Male
count,13017.000000,13017.000000,13017.000000,13017.000000,12958.000000,3353.000000,223.000000,2161.000000,9016.000000,8959.000000,...,5969.000000,13017.000000,13017.000000,13017.000000,13017.000000,13017.000000,13017.000000,13017.000000,13017.000000,13017.000000
mean,2285.200584,73.892064,73.767220,15.994930,0.535654,1.208225,1.783161,1.195504,2.163598,11.398507,...,1.200700,2.235266,26.768221,26.688868,26.498041,0.213106,0.162096,0.312284,0.436199,0.563801
std,1871.013213,110.533877,6.979685,2.824862,0.655480,0.160972,0.422511,0.227999,2.805879,8.616859,...,0.221538,2.225071,26.646135,26.609785,26.398907,0.409518,0.368553,0.463443,0.495932,0.495932
min,2.000000,2.000000,54.400000,4.000000,0.000000,0.636804,1.095000,0.814555,0.000000,0.000000,...,0.838537,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,631.000000,21.000000,69.500000,14.000000,0.000000,1.109730,1.361250,1.010140,0.000000,5.330000,...,1.018950,0.498289,5.967210,6.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1301.000000,41.000000,73.700000,16.000000,0.000000,1.219870,1.850000,1.114670,1.000000,9.000000,...,1.120310,1.524980,18.262300,18.000000,18.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,4353.000000,116.000000,78.600000,18.000000,1.000000,1.314320,2.127500,1.364980,3.000000,15.000000,...,1.374980,3.091030,37.016400,36.000000,36.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,6094.000000,941.000000,91.400000,20.000000,2.000000,1.753320,2.927500,2.669210,18.000000,70.000000,...,2.025560,11.036300,132.164000,132.000000,126.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
# for now, we will fill missing values with 0s
# replace NaNs with 0s
df.fillna(0);

In [5]:
#############################################
### Make Var for whether have Alz or not ####
#############################################

#currently the diagnosis variables = 1 if has any form of dementia
#want to make dummy variable for if they report having alzheimers (not any other type)

alz_at_moment = []
# for each observation in the dataset
for observation in range(len(df.index)):
    if df['DX_bl'][observation] == 'AD':
        alz_at_moment.append(1)
    else:
        alz_at_moment.append(0)
df['alz_at_moment'] = alz_at_moment

In [6]:
#############################
## Make Predictor Variable ##
#############################

# since data is longitudinal, will make a "did they ever get alzheimers" variable
# make a list of the alzheimer dummy values for that patient
#[visit for visit in list(df['DX_Dementia'].groupby(df['PTID']))[1]]
got_alz = []
#for every unique patient
for name, group in df.groupby('PTID'):
    # list of patient's alzheimer dummy values (since longitudinal)
    unique_patient_alz = group['alz_at_moment'].values
    # get 'max' of alzheimers dummy
    # therefore those with max = 1 have reported getting alzheimers in their life
    # those with max = 0 never got alzheimers
    got_alz = got_alz + [max(unique_patient_alz)] * len(unique_patient_alz)

#add it to the dataframe
df['got_alz'] = got_alz

In [7]:
df['Years_bl']

0        0.000000
1        0.000000
2        0.498289
3        0.999316
4        1.998630
5        0.000000
6        0.479124
7        1.015740
8        1.511290
9        3.028060
10       0.000000
11       0.501027
12       0.993840
13       1.998630
14       3.008900
15       0.000000
16       0.503765
17       0.974675
18       1.456540
19       3.025330
20       0.000000
21       0.517454
22       0.000000
23       0.498289
24       0.000000
25       0.492813
26       0.996578
27       1.990420
28       0.000000
29       0.490075
           ...   
12987    0.000000
12988    0.000000
12989    0.618754
12990    2.193020
12991    0.000000
12992    0.465435
12993    2.967830
12994    0.000000
12995    2.384670
12996    0.000000
12997    2.031490
12998    0.000000
12999    0.465435
13000    1.995890
13001    0.000000
13002    0.517454
13003    2.012320
13004    0.000000
13005    0.498289
13006    2.017800
13007    0.000000
13008    0.484600
13009    2.017800
13010    0.000000
13011    2

In [9]:
######################################
### Make Age at Exam Date Variable ###
######################################

age_at_examdate = []
# for each observation in the dataset
for observation in range(len(df.index)):
    # age at exam date is 'age at base line' + 'years from baseline'
    age_at_examdate.append(df['AGE'][observation] + df['Years_bl'][observation])
df['age_at_examdate'] = age_at_examdate

#Now we can get rid of the 'years since baseline' variable


In [11]:
###########################################
### Make Training and test data frames ####
###########################################

np.random.seed(9001)
msk = np.random.rand(len(df)) < 0.5
df_train = df[msk]
df_test = df[~msk]

In [13]:
# proportion of observations in training set that do not have alzheimers
# not alz / (alz + not alz)
# use frequency table
# http://hamelg.blogspot.com/2015/11/python-for-data-analysis-part-19_17.html
pd.crosstab(index=df_train['got_alz'], columns="count")['count'][0] / (pd.crosstab(index=df_train['got_alz'], columns="count")['count'][0]+pd.crosstab(index=df_train['got_alz'], columns="count")['count'][1])


0.87689249120660651

In [ ]:
#############################################################
## scaling test set and training set continuous variables ###
#############################################################
numerical_columns = []
for var in numerical_columns:
    mean = df_train[var].mean()
    std = df_train[var].std()

    df_train[var] = (df_train[var] - mean)/std
    df_test[var] = (df_test[var] - mean)/std

In [ ]:
########################################################
### Prediction Model: Risk of Developing Alzheimers ####
########################################################

# Build a classification model that classifies patients for eventually developing alz or not

# This is an unbalanced dataset!!!!!!

#y_train_risk = df_train['got_alz'].values
#X_train_risk = df_train[risk_predictors].values

#y_test_risk = df_test['got_alz'].values
#X_test_risk = df_test[risk_predictors].values

## will try 4 different models and compare to decide which is best:
## 1. logistic regression
## 2. LDA/QDA
## 3. Decision Trees

# will not k-NN as it is inefficient for a big number of observations

In [ ]:
######################################
### Multinomial Logistic Regression ##
######################################

#add a penalty to prevent overfitting

#logit_model = LogisticRegressionCV(random_state = 123, penalty='l2')
#logit_model.fit(X_train_classif, y_train_classif)
#predict_Logistic_train = logit_model.predict(X_train_classif)
#predict_Logistic_test = logit_model.predict(X_test_classif)

In [ ]:
######################################
### Linear Discriminant Analysis #####
######################################

#lda = da.LinearDiscriminantAnalysis()
#lda.fit(X_train_classif, y_train_classif)
#predict_LDA_train = lda.predict(X_train_classif)
#predict_LDA_test = lda.predict(X_test_classif)

In [ ]:
#########################################
### Quadratic Discriminant Analysis #####
#########################################

#qda = QuadraticDiscriminantAnalysis()
#qda.fit(X_train_classif, y_train_classif)
#predict_QDA_train = qda.predict(X_train_classif)
#predict_QDA_test = qda.predict(X_test_classif)

In [ ]:
#################################
### Decision Trees Analysis #####
#################################
#code from midterm solutions

#dec_tree = DecisionTreeClassifier()
#depths = {'max_depth': list(range(2,15))}
#dec_tree_grid = sk.model_selection.GridSearchCV(dec_tree, depths, cv=5, scoring='roc_auc')
#dec_tree_grid.fit(X_train_classif, y_train_classif)
#best_depth = max([(s,d) for s,d in zip(dec_tree_grid.cv_results_['mean_test_score'],list(range(2,15)))],key=lambda x: x[0])[1]
#dec_tree_best = DecisionTreeClassifier(max_depth=best_depth).fit(X_train_classif, y_train_classif)
#predict_Tree_train = tree_model.predict(X_train_classif)
#predict_Tree_test = tree_model.predict(X_test_classif)

In [ ]:
##################################
### Summary Scores of Models #####
##################################
# on the train and test set

# Here we use the ROC AUC score because the data is unbalanced


## SHOULD THE PREDICTIONS BE PREDICT_PROBA INSTEAAD OF PREDICT????????

print('Logistic Regression model accuracy for training set:', sk.metrics.roc_auc_score(y_train, predict_Logistic_train))
print('Logistic Regression model accuracy for test set:', sk.metrics.roc_auc_score(y_test, predict_Logistic_test))

print('LDA model accuracy for training set:', sk.metrics.roc_auc_score(y_train, predict_LDA_train))
print('LDA model accuracy for test set:', sk.metrics.roc_auc_score(y_test, predict_LDA_test), '\n')

print('QDA model accuracy for training set:', sk.metrics.roc_auc_score(y_train, predict_QDA_train))
print('QDA model accuracy for test set:', sk.metrics.roc_auc_score(y_test, predict_QDA_test), '\n')

print('Decision Tree model (depth ' + str(best_depth) + ') accuracy for training set:', sk.metrics.roc_auc_score(y_train, predict_Tree_train))
print('Decision Tree model (depth ' + str(best_depth) + ') accuracy for test set:', sk.metrics.roc_auc_score(y_test, predict_Tree_test))
